In [1]:
# Import the dependencies
import pandas as pd
import gmaps
import requests

# Import the API key
from config import g_key
from config import key

In [2]:
# Store the CSV into a DataFrame
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Upernavik,GL,2021-01-19 02:13:58,72.7868,-56.1549,7.07,78,10,7.83
1,1,Niort,FR,2021-01-19 02:09:18,46.3333,-0.3333,34.00,88,96,8.50
2,2,Luderitz,NaN,2021-01-19 02:13:58,-26.6481,15.1594,64.92,84,49,11.97
3,3,Rikitea,PF,2021-01-19 02:13:58,-23.1203,-134.9692,78.48,74,72,14.41
4,4,Saint-Philippe,RE,2021-01-19 02:13:58,-21.3585,55.7679,78.80,69,0,4.61


In [3]:
# Get data types
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use Google API key
gmaps.configure(api_key=g_key)
gmaps.configure(api_key=key)

In [5]:
# Get the maximum temperature
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
    
# Or add [max(temp, 0) for temp in max_temp] right in the code below

In [6]:
# Heatmap of temperature
# Get the latitude and longitude
locations = city_data_df[["Lat", "Lng"]]

# Get the maximum temperature
# max_temp = city_data_df["Max Temp"]

# Assign the figure variable
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Assign the heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights=temps, max_intensity=300, point_radius=4, dissipating=False)

# Add the heatmap layer
fig.add_layer(heat_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
wind_speed = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind_speed, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Ask the customer to add a min and max temperature value
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [10]:
# Filter the dataset to find the cities that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,Rikitea,PF,2021-01-19 02:13:58,-23.1203,-134.9692,78.48,74,72,14.41
4,4,Saint-Philippe,RE,2021-01-19 02:13:58,-21.3585,55.7679,78.80,69,0,4.61
7,7,Santa Rosa,PH,2021-01-19 02:13:59,14.3122,121.1114,82.00,74,75,2.30
8,8,Faanui,PF,2021-01-19 02:13:59,-16.4833,-151.7500,80.85,77,4,15.46
11,11,Maceio,BR,2021-01-19 02:13:59,-9.6658,-35.7353,73.40,94,40,4.61
13,13,Panguna,PG,2021-01-19 02:13:59,-6.3164,155.4848,83.52,61,85,3.27
16,16,Georgetown,MY,2021-01-19 02:13:59,5.4112,100.3354,78.80,78,20,2.30
19,19,San Policarpo,PH,2021-01-19 02:14:00,12.1791,125.5072,78.39,90,100,15.95
23,23,Isangel,VU,2021-01-19 02:14:00,-19.5500,169.2667,87.80,74,75,9.22
32,32,Hithadhoo,MV,2021-01-19 02:14:01,-0.6000,73.0833,80.17,69,1,7.67


In [11]:
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Country       0
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [12]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Rikitea,PF,78.48,-23.1203,-134.9692,
4,Saint-Philippe,RE,78.80,-21.3585,55.7679,
7,Santa Rosa,PH,82.00,14.3122,121.1114,
8,Faanui,PF,80.85,-16.4833,-151.7500,
11,Maceio,BR,73.40,-9.6658,-35.7353,
13,Panguna,PG,83.52,-6.3164,155.4848,
16,Georgetown,MY,78.80,5.4112,100.3354,
19,San Policarpo,PH,78.39,12.1791,125.5072,
23,Isangel,VU,87.80,-19.5500,169.2667,
32,Hithadhoo,MV,80.17,-0.6000,73.0833,


In [13]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [14]:
# Iterate through the DataFrame
for index, row in hotel_df.iterrows():
    
    # Get the latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [16]:
# Add a heatmap of temperature for the vacation spots
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))